**Libraries**

In [73]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from hurst import compute_Hc
import statsmodels.api as sm


**Variables**

In [74]:
cryptos = ['BTC-USD', 'ETH-USD', 'USDT-USD', 'BNB-USD', 'SOL-USD', 'XRP-USD', 'USDC-USD',
            'STETH-USD', 'ADA-USD', 'AVAX-USD', 'DOGE-USD', 'DOT-USD', 'WTRX-USD',
              'TRX-USD', 'MATIC-USD', 'LINK-USD', 'TON11419-USD', 'WBTC-USD','ICP-USD','SHIB-USD']

start_date = datetime(2022,11,1)
end_date = datetime(2023,11,2)

my_cryptos = []
stationaries = []
results = {'Names' : list(set()),'P_values' : [],'Weights' : [],'Hurst Exponent': [],'Half-Life' : []}


**Resampling 1h to 4h**

In [75]:
## source : https://stackoverflow.com/questions/74479906/how-to-get-aggregate-4hour-bars-historical-stock-data-using-yfinance-in-python

def Resample(crypto_tickerDf):
    resampled_df = crypto_tickerDf.resample('4H').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })
    return resampled_df

In [76]:
for i in range(len(cryptos)):
    crypto_tickerDf = yf.Ticker(cryptos[i])
    crypto_tickerDf = crypto_tickerDf.history(interval='1h', start=start_date, end=end_date)
    resampled_df = Resample(crypto_tickerDf)
    # resampled_df.replace([np.inf, -np.inf], np.nan).dropna()
    my_cryptos.append(resampled_df['Close'])

**Commented Johansen Test**

In [77]:
# result = coint_johansen(df[['a', 'b', 'c']], det_order=0, k_ar_diff=1)

# # Get the trace statistics
# trace_stat = result.lr1

# # Get the critical values for the trace statistics
# trace_crit_vals = result.cvt

# # Infer the p-value for the trace statistics
# for i in range(len(trace_stat)):
#     if trace_stat[i] > trace_crit_vals[i, 2]:
#         print(f"For r <= {i}, the trace statistic is {trace_stat[i]:.2f} and the p-value is < 0.01")
#     elif trace_stat[i] > trace_crit_vals[i, 1]:
#         print(f"For r <= {i}, the trace statistic is {trace_stat[i]:.2f} and the p-value is < 0.05")
#     elif trace_stat[i] > trace_crit_vals[i, 0]:
#         print(f"For r <= {i}, the trace statistic is {trace_stat[i]:.2f} and the p-value is < 0.1")
#     else:
#         print(f"For r <= {i}, the trace statistic is {trace_stat[i]:.2f} and the p-value is >= 0.1")

**ADF Test**

In [78]:
def modified_adf_test(series):
    adf = adfuller(series)
    p_value = adf[1]
    if p_value <= 0.05 :
      return True,p_value
    return False,p_value

**Johansen Test**

In [79]:
## source : https://blog.quantinsti.com/johansen-test-cointegration-building-stationary-portfolio/

def johansen_test(i,j):
    my_cryptos[i] = my_cryptos[i].replace([np.inf, -np.inf], np.nan).dropna()
    my_cryptos[j] = my_cryptos[j].replace([np.inf, -np.inf], np.nan).dropna()
    data = pd.concat([my_cryptos[i],my_cryptos[j]], axis=1)
    try:
        result = coint_johansen(data.values, det_order=0, k_ar_diff=1)
        weights = result.evec
        result_series = 0
        
        for i in range(2):
            result_series += np.dot(data.values, weights[:, i])
        stationary,p_value = modified_adf_test(result_series)
        if stationary:
            return weights,p_value,result_series
        else:
            return None, None, None
    except np.linalg.LinAlgError:
        # print(f"SVD did not converge for {cryptos[i]} and {cryptos[j]}. Skipping...")
        return None, None,None
    

**Part1**

In [80]:
for i in range(len(my_cryptos)):
    for j in range(i+1,len(my_cryptos)):
        if len(stationaries) < 10:
            weights,p_value,result_series = johansen_test(i,j)
            if weights is not None and p_value is not None:
                stationaries.append(result_series)
                results['Names'].append((cryptos[i],cryptos[j]))
                results['P_values'].append(p_value)
                results['Weights'].append(weights)
        else:
            break


In [81]:
for i in range(len(results['Names'])):
    print(f'Name : {results["Names"][i]}, P-value : {results["P_values"][i]}, Weights : {results["Weights"][i]}')


Name : ('BTC-USD', 'USDT-USD'), P-value : 0.04033124126850145, Weights : [[ 1.58147716e-05 -2.10824405e-04]
 [-9.76506126e+02  2.20993116e+01]]
Name : ('BTC-USD', 'USDC-USD'), P-value : 0.0006447082975304632, Weights : [[ 1.15396871e-05 -2.10397878e-04]
 [-2.61963535e+02 -2.36971319e-01]]
Name : ('BTC-USD', 'DOGE-USD'), P-value : 0.009572761863156639, Weights : [[7.32694857e-05 2.23245477e-04]
 [8.60047187e+01 1.24590296e+01]]
Name : ('BTC-USD', 'WBTC-USD'), P-value : 0.01134791423265375, Weights : [[ 3.38758538e-03 -2.04547968e-04]
 [-3.39888887e-03 -5.88028150e-06]]
Name : ('ETH-USD', 'USDT-USD'), P-value : 0.016250983309399544, Weights : [[ 6.05402788e-04 -4.32229038e-03]
 [-9.81083415e+02  6.00511438e+00]]
Name : ('ETH-USD', 'USDC-USD'), P-value : 0.00034264109824244714, Weights : [[ 1.32995911e-04 -4.31946141e-03]
 [-2.61785651e+02  1.80818868e+00]]
Name : ('ETH-USD', 'DOGE-USD'), P-value : 0.006332303287574753, Weights : [[ 1.04575543e-03  4.30974119e-03]
 [ 7.99226264e+01 -7.004

**Part2 - Hurst Exponent**

In [82]:
## source : https://github.com/Mottl/hurst
def HurstExponent(timeSeries):
    H, c, data = compute_Hc(timeSeries)
    # print(f"Hurst exponent: {H:.4f}")
    # print(f"Constant: {c:.4f}")
    return H,c

In [83]:
for ts in range(len(stationaries)):
    hurst_exponent,constant = HurstExponent(stationaries[ts])
    results['Hurst Exponent'].append(hurst_exponent)
    print(f'Hurst Exponent : {hurst_exponent} \t Constant : {constant}')
    print('-------------------------------------')

Hurst Exponent : 0.4696510141229028 	 Constant : 1.2229280771613853
-------------------------------------
Hurst Exponent : 0.4809935827435661 	 Constant : 1.11280141106601
-------------------------------------
Hurst Exponent : 0.4728421852659293 	 Constant : 1.4556742229216872
-------------------------------------
Hurst Exponent : 0.39895541055974704 	 Constant : 1.2465501872472171
-------------------------------------
Hurst Exponent : 0.4668509688457538 	 Constant : 1.233781799571892
-------------------------------------
Hurst Exponent : 0.4670359510434491 	 Constant : 1.2816286328288455
-------------------------------------
Hurst Exponent : 0.4702840485591979 	 Constant : 1.4575064078477173
-------------------------------------
Hurst Exponent : 0.4769990013509742 	 Constant : 1.1306296583890694
-------------------------------------
Hurst Exponent : 0.460413053972439 	 Constant : 1.3467711586569087
-------------------------------------
Hurst Exponent : 0.4755526210549911 	 Constant : 

**Part3 - half-life**

In [84]:
## source : https://quant.stackexchange.com/questions/25086/calculating-half-life-of-mean-reverting-series-with-python
## source2 : https://github.com/cantaro86/Financial-Models-Numerical-Methods/blob/master/6.1%20Ornstein-Uhlenbeck%20process%20and%20applications.ipynb

def half_life(timeseries):
    # Calculate the lagged time series
    lagged_timeseries = np.roll(timeseries, 1)
    lagged_timeseries[0] = 0
    delta_timeseries = timeseries - lagged_timeseries
    delta_timeseries[0] = 0

    # Add a constant to the lagged timeseries
    lagged_timeseries = sm.add_constant(lagged_timeseries[1:])

    # Use the OLS function to calculate theta
    model = sm.OLS(delta_timeseries[1:], lagged_timeseries)
    results = model.fit()
    theta = results.params

    # Calculate and return the half life
    return -np.log(2) / theta[1]

In [85]:
for i in range(len(stationaries)):
    results['Half-Life'].append(half_life(stationaries[i]))
    print(results['Names'][i] ," : ", half_life(stationaries[i]))

('BTC-USD', 'USDT-USD')  :  5.945232677165996
('BTC-USD', 'USDC-USD')  :  6.387315540910477
('BTC-USD', 'DOGE-USD')  :  73.10259818825904
('BTC-USD', 'WBTC-USD')  :  2.552096998420777
('ETH-USD', 'USDT-USD')  :  5.6327316557820675
('ETH-USD', 'USDC-USD')  :  6.41194222590925
('ETH-USD', 'DOGE-USD')  :  69.62349053722
('USDT-USD', 'BNB-USD')  :  5.333823154374098
('USDT-USD', 'SOL-USD')  :  5.8640801234808615
('USDT-USD', 'XRP-USD')  :  5.665302582212841


**LAST PART**

In [86]:
for i in range(len(results['Names'])):
    print(f'Name :{results["Names"][i]}   P-value :{results["P_values"][i]}  Weights :{results["Weights"][i]}   Hurst Exponent :{results["Hurst Exponent"][i]}   Half-Life :{results["Half-Life"][i]}\n')

Name :('BTC-USD', 'USDT-USD')   P-value :0.04033124126850145  Weights :[[ 1.58147716e-05 -2.10824405e-04]
 [-9.76506126e+02  2.20993116e+01]]   Hurst Exponent :0.4696510141229028   Half-Life :5.945232677165996

Name :('BTC-USD', 'USDC-USD')   P-value :0.0006447082975304632  Weights :[[ 1.15396871e-05 -2.10397878e-04]
 [-2.61963535e+02 -2.36971319e-01]]   Hurst Exponent :0.4809935827435661   Half-Life :6.387315540910477

Name :('BTC-USD', 'DOGE-USD')   P-value :0.009572761863156639  Weights :[[7.32694857e-05 2.23245477e-04]
 [8.60047187e+01 1.24590296e+01]]   Hurst Exponent :0.4728421852659293   Half-Life :73.10259818825904

Name :('BTC-USD', 'WBTC-USD')   P-value :0.01134791423265375  Weights :[[ 3.38758538e-03 -2.04547968e-04]
 [-3.39888887e-03 -5.88028150e-06]]   Hurst Exponent :0.39895541055974704   Half-Life :2.552096998420777

Name :('ETH-USD', 'USDT-USD')   P-value :0.016250983309399544  Weights :[[ 6.05402788e-04 -4.32229038e-03]
 [-9.81083415e+02  6.00511438e+00]]   Hurst Expon

In [87]:
df = pd.DataFrame()

df['name1'] = [name[0] for name in results['Names']]
df['name2'] = [name[1] for name in results['Names']]

df['P_values'] = results['P_values']
df['Weights'] = results['Weights']
df['Hurst Exponent'] = results['Hurst Exponent']
df['Half-Life'] = results['Half-Life']

df.to_csv('results.csv', index=False)